<a href="https://colab.research.google.com/github/rjohns2991/job-screening-exercises/blob/main/Wintrust_Bank_Data_Quality_Analyst_Screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

/*

### Question: Show all steps' result in one table and use recursive CTE (With ... as) to write the query.

**In a retail database, we need to ensure data quality by performing the following tasks:**

1. Identify and clean customer phone numbers by removing any non-numeric characters and standardizing them to a specific format.

2. Check for duplicates in the customer records based on `PhoneNumber`, flagging them for review.

3. Calculate the total purchase amount for each customer, ensuring that we only consider unique purchases (i.e., if a customer made multiple purchases of the same item, it should be counted only once).

4. Return a summary of customers with their total purchases, Flag them whose total purchase amount exceeds a specified threshold (which is $500).

5. Identify customers who have not made any purchases in the last 12 months. Label them as Inactive.

*/

---


In [ ]:
# CREATING THE TABLE
import sqlite3
import pandas as pd


conn = sqlite3.connect('test.db')
print("Opened database successfully")

conn.execute('''
CREATE TABLE IF NOT EXISTS Interview_Customers(
    CustomerID INT PRIMARY KEY,
    Name VARCHAR(100),
    Email VARCHAR(100) UNIQUE,
    PhoneNumber VARCHAR(20),
    RegistrationDate DATE);''')
conn.commit()
print("Table created successfully")

conn.execute('''
INSERT INTO Interview_Customers (CustomerID, Name, Email, PhoneNumber, RegistrationDate)
VALUES
(1, 'Alice Johnson', 'alice@gmail.com', '(123) 456-7890', '2023-01-10'),
(2, 'Bob Smith', 'bob@yahoo.com', '123-456-7890', '2023-02-15'),
(3, 'Charlie Brown', 'charlie@company.com', '1234567890', '2023-03-20'),
(4, 'David Wilson', 'david@gmail.com', '(987) 654-3210', '2023-04-25'),
(5, 'Eva Green', 'eva@gmail.com', '987.654.3210', '2023-05-30'),
(6,    'Desa Sweet', 'dsweet@gmail.com',  '322.654.32-10', '2022-05-30'),
(7,    'Sena Sweet', 'ssweet@gmail.com',  '312.654.32-10', '2024-05-30');''')
conn.commit()

conn.execute('''
CREATE TABLE Interview_Purchases (
    PurchaseID INT PRIMARY KEY,
    CustomerID INT,
    Item VARCHAR(100),
    Amount DECIMAL(10, 2),
    PurchaseDate DATE,
    FOREIGN KEY (CustomerID) REFERENCES Interview_Customers(CustomerID)
);''')
conn.commit()

conn.execute('''
INSERT INTO Interview_Purchases (PurchaseID, CustomerID, Item, Amount, PurchaseDate)
VALUES
(1, 1, 'Laptop', 999.99, '2023-06-01'),
(2, 1, 'Mouse', 25.00, '2023-06-01'),
(3, 2, 'Keyboard', 50.00, '2023-07-01'),
(4, 3, 'Laptop', 999.99, '2023-08-01'),
(5, 1, 'Mouse', 25.00, '2023-06-01'),
(6, 4, 'Monitor', 300.00, '2023-10-01'),
(7, 5, 'Desk', 200.00, '2023-11-01'),
(8, 5, 'Chair', 150.00, '2023-11-15'),
(9, 2, 'Laptop', 999.99, '2023-12-01'),
(10, 6, 'Mouse', 25.00, '2022-05-30'),
(11, 7, 'Desk', 200.00, '2024-05-30');''')
conn.commit()

conn.close()

In [ ]:
query = '''
WITH
CleanedPhones AS (
    SELECT
        CustomerID,
        Name,
        Email,
        REPLACE(REPLACE(REPLACE(REPLACE(PhoneNumber, '(', ''), ')', ''), '-', ''), '.', '') AS CleanPhoneNumber,
        RegistrationDate
    FROM Interview_Customers
),
Duplicates AS (
    SELECT
        CleanPhoneNumber,
        COUNT(*) AS DuplicateCount
    FROM CleanedPhones
    GROUP BY CleanPhoneNumber
    HAVING COUNT(*) > 1
),
UniquePurchases AS (
    SELECT
        CustomerID,
        Item,
        MAX(Amount) AS Amount
    FROM Interview_Purchases
    GROUP BY CustomerID, Item
),
TotalPurchases AS (
    SELECT
        c.CustomerID,
        c.Name,
        c.Email,
        c.CleanPhoneNumber,
        c.RegistrationDate,
        COALESCE(SUM(u.Amount), 0) AS TotalPurchaseAmount
    FROM CleanedPhones c
    LEFT JOIN UniquePurchases u
    ON c.CustomerID = u.CustomerID
    GROUP BY c.CustomerID, c.Name, c.Email, c.CleanPhoneNumber, c.RegistrationDate
),
FlaggedPurchases AS (
    SELECT
        CustomerID,
        Name,
        Email,
        CleanPhoneNumber,
        RegistrationDate,
        TotalPurchaseAmount,
        CASE
            WHEN TotalPurchaseAmount > 500 THEN 'Flagged'
            ELSE 'Normal'
        END AS PurchaseFlag
    FROM TotalPurchases
),
InactiveCustomers AS (
    SELECT
        fp.CustomerID,
        fp.Name,
        fp.Email,
        fp.CleanPhoneNumber,
        fp.RegistrationDate,
        fp.TotalPurchaseAmount,
        fp.PurchaseFlag,
        CASE
            WHEN NOT EXISTS (
                SELECT 1
                FROM Interview_Purchases p
                WHERE p.CustomerID = fp.CustomerID
                  AND p.PurchaseDate >= DATE('now', '-1 year')
            ) THEN 'Inactive'
            ELSE 'Active'
        END AS Status
    FROM FlaggedPurchases fp
)
SELECT
    CustomerID,
    Name,
    Email,
    CleanPhoneNumber AS PhoneNumber,
    RegistrationDate,
    TotalPurchaseAmount,
    PurchaseFlag,
    Status
FROM InactiveCustomers
ORDER BY CustomerID;
'''

# Execute the query
result = pd.read_sql_query(query, conn)

# Display the result
result


,CustomerID,Name,Email,PhoneNumber,RegistrationDate,TotalPurchaseAmount,PurchaseFlag,Status
0,1,Alice Johnson,alice@gmail.com,123 4567890,2023-01-10,1024.99,Flagged,Inactive
1,2,Bob Smith,bob@yahoo.com,1234567890,2023-02-15,1049.99,Flagged,Active
2,3,Charlie Brown,charlie@company.com,1234567890,2023-03-20,999.99,Flagged,Inactive
3,4,David Wilson,david@gmail.com,987 6543210,2023-04-25,300.00,Normal,Inactive
4,5,Eva Green,eva@gmail.com,9876543210,2023-05-30,350.00,Normal,Inactive
5,6,Desa Sweet,dsweet@gmail.com,3226543210,2022-05-30,25.00,Normal,Inactive
6,7,Sena Sweet,ssweet@gmail.com,3126543210,2024-05-30,200.00,Normal,Active


---

I was extended an invitation for a third interview following this exercise but chose to decline, as I had received more competitive offers in terms of compensation and benefits from highly sought-after employers.